# Greedy

- problem: jumlah minimal tim dalam constraint yg diberikan
- constraint: is activity compatible with some team and deadline?

## todo
- fix any such team

In [1]:
import pandas as pd
from pprint import pprint

In [75]:
"""
https://docs.google.com/spreadsheets/d/1KxWTopBOX0lhMYJ0ij3u7sYs1s871_Vk/edit
"""

class Greedy():
    a = 1
    data = None
    df = None
    teamWork = {}
    teamCount = 0
    maxDayPerTeam = 30*3
    selectTeam = 0
    
    def __init__(self):
        pass

    def load(self, fname):
        self.df = pd.read_csv(fname)
        return self
    
    def loadSet(self, fname):
        self.load(fname)
        self.data = self.df
        return self
    
    def reset(self):
        self.teamWork = {}
        return self

    def sortBy(self, col):
        self.data = self.df.sort_values(col)
        return self
    
    def setMaxDayPerTeam(self, maxd):
        self.maxDayPerTeam = maxd
        return self

    # df compatible data
    def setData(self, data):
        self.data = data
        return self
    
    def addWorkToTeam(self, key, data):
        self.teamWork[key]['works'].append(data)
        current = self.teamWork[key]['totalDay']
        self.teamWork[key]['totalDay'] = current + data['dur']
        return self
    
    def isCompatibleToAnyTeam(self, data):
        compat = False
        for t in g.teamWork.keys():
            lastWork = g.teamWork[t]['works'][-1]

            if lastWork['finish'] < data['start']:
                self.selectTeam = t
                compat = True
                break

        return compat

    # df compatible data
    def addTeam(self, key, data):
        self.teamWork[key] = data
        return self

    def isFeasible(self, idx, data):
        """
        data = {
            "team": self.teamCount,
            "work": d,
        }
        """
        try:
            c1 = not len(self.teamWork.keys()) < 1
            c2 = self.maxDayPerTeam >= self.teamWork[idx]['totalDay']
            c3 = self.isCompatibleToAnyTeam(data['work'])

            # todo : if compatible with some team
        except:
            return False
        cond = c1 & c2 & c3
        return cond
    
    def doGreedy(self):
        res = {}
        # for i,d in enumerate(self.data):
        for i,d in self.data.iterrows():
            fdata = {
                "team": self.teamCount,
                "work": d,
            }
            teamidx = str(self.teamCount)
            if self.isFeasible(teamidx, fdata):
                # add to existing teamwork
                self.addWorkToTeam(self.selectTeam, d)
            else:
                # add new data to teamwork
                idx = str(self.teamCount+1)
                self.addTeam(idx, {
                    "works":[],
                    "totalDay": 0,
                })
                self.addWorkToTeam(idx, d)
                self.teamCount += 1
#             print(self.teamWork)
        return res
      
    def showResult(self):
        for t in g.teamWork.keys():
            wt = g.teamWork[t]['works']
            ld = len(wt)
            print(t, f"{ld}", ["{} - {}".format(w['start'], w['finish']) for w in wt])

g = Greedy()
# fname = "const-greedy.csv"
fname = "https://gist.github.com/adib-enc/4408230ef5f91c00dae66406830348c0/raw/15cbdc1bbd67b54b07f1a9dcfd6727b1fc7cf43a/const-greedy.csv"
g.loadSet(fname)
g.data = g.data[0:6]
g.sortBy("start")

In [76]:
g.data

,no,dur,dreward,reward total,cost,start,finish
0,1,24,2000,48000,1200,1,25
12,13,24,2000,48000,1800,9,33
9,10,33,2000,66000,320,11,44
5,6,24,2000,48000,18000,41,65
13,14,18,2000,36000,2200,48,66
4,5,30,2000,60000,10000,67,97
7,8,24,2000,48000,120,78,102
17,18,18,2000,36000,2200,88,106
11,12,30,2000,60000,1000,101,131
16,17,24,2000,48000,1800,111,135


In [77]:
g.setMaxDayPerTeam(365)
# g.setMaxDayPerTeam(120)
result = g.reset().doGreedy()
# print(g.data)
teamCount = len(g.teamWork.keys())
print(f"constraint: {g.maxDayPerTeam} hari per tim")
print(f"minimal tim: {teamCount}")

constraint: 365 hari per tim
minimal tim: 5


In [81]:
g.showResult()

1 6 ['1 - 25', '41 - 65', '67 - 97', '101 - 131', '141 - 159', '166 - 191']
2 6 ['9 - 33', '48 - 66', '78 - 102', '111 - 135', '153 - 169', '174 - 194']
3 4 ['11 - 44', '88 - 106', '117 - 142', '155 - 188']
4 1 ['125 - 155']
5 1 ['130 - 150']


In [79]:
g.data.shape[0]

18

In [80]:
# https://www.toptal.com/developers/hastebin/ivecicixut.yaml
"""
function optimasi(){
    
    dur #jumlah hari yang diperlukan untuk pengerjaan
    cost #biaya yang diperlukan
    costcrash #biayapengahancuran 
    expdaycrash #lamanya waktu crash
    dailyreward #reward
    timeleft = 4*5*6  = 120 #waktu kerja selama 6 bulan
    task = #jumlah task

    #time
    #reward = 0
    #task = 0
    if(dur > 0 and timeleft > 0){
        timeleft = timeleft - dur;
        if(timeleft > 0 and expdaycrash > 0){
            timeleft = timeleft - expdaycrash;
            if(timeleft > 0){
                task = task + 1; #initiate task
            }
        }
    }

    #costreward
    if(cost != 0){
        totalcost = cost;
        if(costcrash != 0){
             totalcost = totalcost + costcrash
        }
        reward = dailyreward - totalcost;
    }

    print(task,reward,)
}
"""
# a
"fun"

'fun'